In [11]:
#Import drive
from google.colab import drive
#Mount Google Drive
ROOT="/content/drive"
drive.mount(ROOT, force_remount=True)

Mounted at /content/drive


In [12]:
%pwd

'/content'

In [13]:
%cd "/content/drive/MyDrive/dlss24/dlss24"

/content/drive/MyDrive/dlss24/dlss24


In [14]:
!git branch

  main
* workplace


In [15]:
!git config --global user.email "tavares.susana@outlook.com"
!git config --global user.name "susanatavares"

In [16]:
!git pull

Already up to date.


In [7]:
%ls

'Class 2 - Lab 28.04 - Basics .ipynb'        'Notes Class 3 - Lab 06.03 - Classes .ipynb'
'Class 3 - Lab 06.03 - Classes .ipynb'       'Notes Class 3 - Lab 06.03 - Data Analysis.ipynb'
'Class 3 - Lab 06.03 - Data Analysis.ipynb'   __pycache__/
 data/                                        README.md
 mymodule.py                                  utils.py
'Notes Class 2 - Lab 28.04 - Basics .ipynb'


# Week 2 - Class 2 - Data Analysis

This notebook have good example exercises to get you going on data analysis

# Coronavirus (COVID-19) Visualization & Prediction  
Coronavirus is a family of viruses that are named after their spiky crown. The novel coronavirus, also known as SARS-CoV-2, is a contagious respiratory virus that first reported in Wuhan, China. On 2/11/2020, the World Health Organization designated the name COVID-19 for the disease caused by the novel coronavirus. This notebook aims at exploring COVID-19 through data analysis and projections.

   Coronavirus Case Data is provided by <a href='https://github.com/CSSEGISandData/COVID-19'>Johns Hopkins University</a>
   <br>Learn more from the <a href='https://www.who.int/emergencies/diseases/novel-coronavirus-2019'>World Health Organization</a>
   <br>Learn more from the <a href='https://www.cdc.gov/coronavirus/2019-ncov'>Centers for Disease Control and Prevention</a>
   <br>Check out map visualizations from  <a href='https://gisanddata.maps.arcgis.com/apps/opsdashboard/index.html#/bda7594740fd40299423467b48e9ecf6'>JHU CCSE Dashboard</a>
   <br>Source code is also on <a href='https://github.com/therealcyberlord'>my Github</a>
   
   
   ```Last update: 1/16/2023 7:41 PM ET. New Updates: Daily report data update for 1/16/23. time series data update for 1/15/23. ```

source: https://www.kaggle.com/code/therealcyberlord/coronavirus-covid-19-visualization-prediction/notebook

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import pandas as pd
import random
import math
import time
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
import datetime
import operator
%matplotlib inline
import warnings

plt.style.use('seaborn-poster')
warnings.filterwarnings("ignore")

In [10]:
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
latest_data = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-15-2023.csv')

In [11]:
confirmed_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,209322,209340,209358,209362,209369,209390,209406,209436,209451,209451
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,334391,334408,334408,334427,334427,334427,334427,334427,334443,334457
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,271441,271448,271463,271469,271469,271477,271477,271490,271494,271496
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,47866,47875,47875,47875,47875,47875,47875,47875,47890,47890
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,105255,105277,105277,105277,105277,105277,105277,105277,105288,105288


In [12]:
confirmed_df.shape

(289, 1147)

In [13]:
def count_elements_with_characters(lst):
    count = 0
    for element in lst:
        if any('a' <= char <= 'z' for char in element):
            count += 1
    return count

In [14]:
def count_non_date_elements(lst):
  from datetime import datetime

  count = 0
  for element in lst:
      try:
          datetime.strptime(element, '%m/%d/%y')
      except ValueError:
          count += 1
  return count


In [15]:
confirmed_cols = confirmed_df.columns
deaths_cols = deaths_df.columns

confirmed = confirmed_df.loc[:, confirmed_cols[4]:]
deaths = deaths_df.loc[:, deaths_cols[4]:]

In [20]:
count_elements_with_characters(confirmed_cols)


4

In [21]:
count_non_date_elements(confirmed)

0

In [22]:
num_dates = len(confirmed.columns)
ck = confirmed.columns
dk = deaths.columns

world_cases = []
total_deaths = []
mortality_rate = []



In [23]:
num_dates

1143

In [24]:
# what is this number?
confirmed[ck[1143-1]].sum()

676570149

**Answer:** is it the total number of covid cases in the dataset?

In [31]:
num_dates = len(confirmed.columns)
ck = confirmed.columns
dk = deaths.columns

world_cases = []
total_deaths = []
mortality_rate = []

# list comprehension of this?
for i in range(num_dates):
    confirmed_sum = confirmed[ck[i]].sum()
    death_sum = deaths[dk[i]].sum()

    world_cases.append(confirmed_sum)
    total_deaths.append(death_sum)

    # calculate rates
    mortality_rate.append(death_sum/confirmed_sum)

In [39]:
confirmed_sum = [confirmed[ck[i]].sum() for i in range(num_dates)]
death_sum = [deaths[dk[i]].sum() for i in range(num_dates)]
mortality = [death_sum[i]/confirmed_sum[i] for i in range(num_dates)]

In [44]:
def daily_increase(data):
    d = []
    for i in range(len(data)):
        if i == 0:
            d.append(data[0])
        else:
            d.append(data[i]-data[i-1])
    return d

def moving_average(data, window_size):
    moving_average = []
    for i in range(len(data)):
        if i + window_size < len(data):
            moving_average.append(np.mean(data[i:i+window_size]))
        else:
            moving_average.append(np.mean(data[i:len(data)]))
    return moving_average

# window size
window = 7

# confirmed cases
world_daily_increase = daily_increase(world_cases)
world_confirmed_avg= moving_average(world_cases, window)
world_daily_increase_avg = moving_average(world_daily_increase, window)

# deaths
world_daily_death = daily_increase(total_deaths)
world_death_avg = moving_average(total_deaths, window)
world_daily_death_avg = moving_average(world_daily_death, window)

In [48]:
days_since_1_22 = np.array([i for i in range(len(ck))]).reshape(-1, 1)
world_cases = np.array(world_cases).reshape(-1, 1)
total_deaths = np.array(total_deaths).reshape(-1, 1)

In [45]:
days_in_future = 10
future_forcast = np.array([i for i in range(len(ck)+days_in_future)]).reshape(-1, 1)
adjusted_dates = future_forcast[:-10]

In [46]:
start = '1/22/2020'
start_date = datetime.datetime.strptime(start, '%m/%d/%Y')
future_forcast_dates = []
for i in range(len(future_forcast)):
    future_forcast_dates.append((start_date + datetime.timedelta(days=i)).strftime('%m/%d/%Y'))

In [49]:
# slightly modify the data to fit the model better (regression models cannot pick the pattern), we are using data from 8/1/22 and onwards for the prediction modeling
days_to_skip = 922
X_train_confirmed, \
X_test_confirmed, \
y_train_confirmed, \
y_test_confirmed = \
train_test_split(days_since_1_22[days_to_skip:], world_cases[days_to_skip:], test_size=0.07, shuffle=False)

In [50]:
X_train_confirmed

array([[ 922],
       [ 923],
       [ 924],
       [ 925],
       [ 926],
       [ 927],
       [ 928],
       [ 929],
       [ 930],
       [ 931],
       [ 932],
       [ 933],
       [ 934],
       [ 935],
       [ 936],
       [ 937],
       [ 938],
       [ 939],
       [ 940],
       [ 941],
       [ 942],
       [ 943],
       [ 944],
       [ 945],
       [ 946],
       [ 947],
       [ 948],
       [ 949],
       [ 950],
       [ 951],
       [ 952],
       [ 953],
       [ 954],
       [ 955],
       [ 956],
       [ 957],
       [ 958],
       [ 959],
       [ 960],
       [ 961],
       [ 962],
       [ 963],
       [ 964],
       [ 965],
       [ 966],
       [ 967],
       [ 968],
       [ 969],
       [ 970],
       [ 971],
       [ 972],
       [ 973],
       [ 974],
       [ 975],
       [ 976],
       [ 977],
       [ 978],
       [ 979],
       [ 980],
       [ 981],
       [ 982],
       [ 983],
       [ 984],
       [ 985],
       [ 986],
       [ 987],
       [ 9

In [51]:

poly = PolynomialFeatures(degree=3)
poly_X_train_confirmed = poly.fit_transform(X_train_confirmed)
poly_X_test_confirmed = poly.fit_transform(X_test_confirmed)
poly_future_forcast = poly.fit_transform(future_forcast)

In [1]:
# polynomial regression
linear_model = LinearRegression(normalize=True, fit_intercept=True)
linear_model.fit(poly_X_train_confirmed, y_train_confirmed)
test_linear_pred = linear_model.predict(poly_X_test_confirmed)
linear_pred = linear_model.predict(poly_future_forcast)
print('MAE:', mean_absolute_error(test_linear_pred, y_test_confirmed))
print('MSE:',mean_squared_error(test_linear_pred, y_test_confirmed))

NameError: name 'LinearRegression' is not defined

In [ ]:
print(linear_model.coef_)

In [ ]:
plt.plot(y_test_confirmed)
plt.plot(test_linear_pred)
plt.legend(['Test Data', 'Polynomial Regression Predictions'])

## Q16

In [ ]:
# https://scikit-learn.org/stable/getting_started.html#fitting-and-predicting-estimator-basics
# Apply a Ramdom Forest Classifier and do the same plot

## Q17

In [ ]:
import pandas as pd
from sklearn.datasets import load_iris

# Load the iris dataset
iris_data = load_iris()

# Create a DataFrame
<your-answer>

# Add the target (species) column to the DataFrame
<your-answer>

# Map target numerical values to species names
<your-answer>




In [ ]:
# Display basic information
print("Basic Information:")

print("\nData Types:")


In [ ]:

print("\nSummary Statistics:")


In [ ]:

# Data Exploration
print("\nMean Values by Species:")



In [ ]:

print("\nMedian Values by Species:")


In [ ]:

print("\nMinimum Values by Species:")


In [ ]:

print("\nMaximum Values by Species:")


In [ ]:

print("\nCount of Species:")


In [ ]:
!git add "Notes Class 3 - Lab 06.03 - Data Analysis.ipynb"

In [2]:
!git status

fatal: not a git repository (or any of the parent directories): .git


In [ ]:
!git commit -m"Forgot to add identifier"

[workplace 1d4f32e] Forgot to add identifier
 2 files changed, 1309 insertions(+)
 create mode 100644 Notes Class 3 - Lab 06.03 - Classes .ipynb
 create mode 100644 Notes Class 3 - Lab 06.03 - Data Analysis.ipynb


In [ ]:
!git push origin workplace

Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 5.04 KiB | 574.00 KiB/s, done.
Total 3 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
remote: 
remote: Create a pull request for 'workplace' on GitHub by visiting:
remote:      https://github.com/susanatavares/dlss24/pull/new/workplace
remote: 
To https://github.com/susanatavares/dlss24.git
 * [new branch]      workplace -> workplace
Branch 'workplace' set up to track remote branch 'workplace' from 'origin'.
